In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
from extract import open_driver, close_driver, wait_for_element
from transforms import parse_record, parse_game_date
from selenium.webdriver.common.by import By
from DML import db_connect, close_db_connect
from sql_queries import summary_table_insert
from psycopg2.extras import execute_batch

In [11]:
def download(url, driver):
    driver.get(url)
    root_element = wait_for_element(source=driver, search_by=By.ID, target="root", unique_element=True)
    data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)
    
    page_jump = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="-pageJump", unique_element=True)
    page_number = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="-totalPages", unique_element=True)
    
    table_headers = get_table_columns(data_table)
    table_data = get_table_rows(data_table, table_headers)
    
    return table_data

In [ ]:
def get_data_table(url):
    driver.get(url)
    root_element = wait_for_element(source=driver, search_by=By.ID, target="root", unique_element=True)
    data_table = wait_for_element(source=root_element, search_by=By.CLASS_NAME, target="rt-table", unique_element=True)
    
    return data_table

In [12]:
def get_table_columns(table_element):
    header_container = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="tableHeaderDiv", unique_element=True)
    header_elements = wait_for_element(source=header_container, search_by=By.CLASS_NAME, target="rt-th")
    header_names = [header.text for header in header_elements]
    
    return header_names

In [ ]:
def get_table_rows(table_element):
    table_rows = wait_for_element(source=table_element, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
    rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
    
    return rows

In [13]:
# def get_table_rows(source, headers):
#     table_rows = wait_for_element(source=source, search_by=By.CLASS_NAME, target="rt-tbody", unique_element=True)
#     rows = wait_for_element(source=table_rows, search_by=By.CLASS_NAME, target="rt-tr")
#     table_data = []
    
#     for row in rows:
#         cells = wait_for_element(source=row, search_by=By.CLASS_NAME, target="rt-td")
#         row_values = [cell.text for cell in cells]
        
#         values_map = list(zip(headers, row_values))
#         map_dict = parse_record(values_map)
        
#         ordered_data_list = [
#             map_dict["Team"],
#             "2021/22",
#             map_dict["game_date"],
#             map_dict["home_game"],
#             map_dict["opponent"],
#             map_dict["W"],
#             map_dict["L"],
#             map_dict["T"],
#             map_dict["OT"],
#             map_dict["P"],
#             map_dict["P%"],
#             map_dict["RW"],
#             map_dict["ROW"],
#             map_dict["S/O Win"],
#             map_dict["GF"],
#             map_dict["GA"],
#             map_dict["PP%"],
#             map_dict["PK%"],
#             map_dict["Net PP%"],
#             map_dict["Net PK%"],
#             map_dict["Shots/GP"],
#             map_dict["GA/GP"],
#             map_dict["FOW%"]
#         ]
        
#         table_data.append(ordered_data_list)
        
#     return table_data

In [14]:
def scrape_page(url):
    driver = open_driver()
    table_data = download(url, driver)
    close_driver(driver)
    
    # call batch upload function
    conn, cur = db_connect()
    execute_batch(cur, summary_table_insert, table_data)
    close_db_connect(conn, cur)

In [15]:
url = "https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20212022&seasonTo=20212022&dateFromSeason&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=100"

In [16]:
scrape_page(url=url)

DatatypeMismatch: ARRAY types timestamp without time zone and boolean cannot be matched
LINE 1: ...evils','2021/22','2021-11-06T00:00:00'::timestamp,false,'SJS...
                                                             ^


In [22]:
class Scraper:
    def __init__(self):
        self.conn, self.cur = db_connect()
    
    def close_db(self):
        close_db_connect(self.conn, self.cur)

In [23]:
scraper = Scraper()

In [24]:
scraper.close_db()

In [1]:
test_dict = {'a': 1, 'b': 2, 'c': 3}

In [2]:
test_dict.keys()

dict_keys(['a', 'b', 'c'])

In [3]:
test_dict.values()

dict_values([1, 2, 3])